In [5]:
import os
os.environ["CUDA_DEVICES_ORDER"]= "PCI_BUS_IS"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

class pde_wan():
    
    def __init__(self, dim=2):
        import numpy as np
        global np
        #
        import time
        global time
        #
        #import tensorflow as tf
        import tensorflow.compat.v1 as tf
        global tf
        tf.disable_v2_behavior()
        #
        import matplotlib.pyplot as plt
        global plt
        #
        from scipy.interpolate import griddata
        global griddata
        #
        self.up= 1.0
        self.low= 0
        
        self.mesh_size= 256
        #
        self.v_layer= 6
        self.v_hidden_size= 50
        self.v_step= 1       
        self.v_rate=0.015   
        self.u_layer= 6
        self.u_hidden_size= 20
        self.u_step= 1
        self.u_rate=0.015
        self.batch_size= 20000 
        self.test_size=  5000
        self.bound_size= 100#(*2*dim)
        self.iteration=  20001
        self.dim= dim 
        self.dir= './problem_weak/'
    def sample_train(self, dm_size, bd_size, dim):      ##dm-domain;bd-boundry;dim-dimension
        low= self.low; up= self.up;
        #*********************************************************
        # collocation points in domain
        x_dm= np.random.uniform(low, up, [dm_size, dim]) ##up=1;low=0;dim=2
        
        #*********************************************************
        # collocation points on boundary
        x_bd_list=[]  #x_boundry
        for i in range(dim):
            x_bound= np.random.uniform(low, up, [bd_size, dim])
            x_bound[:,i]= up
            x_bd_list.append(x_bound)
            x_bound= np.random.uniform(low, up, [bd_size, dim])
            x_bound[:,i]= low
            x_bd_list.append(x_bound)
        x_bd= np.concatenate(x_bd_list, axis=0)
        
        #*********************************************************
        int_dm= (up-low)**dim        ##integrate domain
        #*********************************************************
        f_dm=np.ones([dm_size,1])
        for i in range(dm_size):
          f_dm[i]=1
      
        #*********************************************************
        # u(x) on boundary
        x= np.reshape(x_bd[:,0], [-1,1])
        y= np.reshape(x_bd[:,1], [-1,1])
        u_bd=np.ones([4*bd_size,1])
        for i in range(4*bd_size):
          u_bd[i]=-((x[i]**2)+(y[i]**2))/4

        #*********************************************************
        x_dm= np.float32(x_dm)  
        x_bd= np.float32(x_bd)
        int_dm= np.float32(int_dm)
        f_dm= np.float32(f_dm)
        u_bd= np.float32(u_bd)
        return(x_dm, f_dm, x_bd, u_bd, int_dm)      ##int_dm=integrand domain


    def sample_test(self, test_size, dim):
        up= self.up; low= self.low;
        #**********************************************************
        x_mesh= np.linspace(low, up, self.mesh_size)
        mesh= np.meshgrid(x_mesh, x_mesh)
        x= np.reshape(mesh[0], [-1,1])
        y= np.reshape(mesh[1], [-1,1])
        x_dm= np.concatenate((x,y), 1) 
        #***********************************************************
        u_dm=np.ones([65536,1])
        for i in range(65536):
          u_dm[i]=-((x_dm[i,0]**2)+(x_dm[i,1]**2))/4
        #***********************************************************
        x_dm= np.float32(x_dm)
        u_dm= np.float32(u_dm)
        return(mesh, x_dm, u_dm)


    def DNN_u(self, x_in, out_size, name, reuse):
        h_size= self.u_hidden_size
        with tf.variable_scope(name, reuse= reuse):   ##可以让变量有相同命名
            hi= tf.layers.dense(x_in, h_size, activation= tf.nn.tanh,    ##tf.layers.dense表示添加一个层，全连接层
                                name='input_layer')
            hi= tf.layers.dense(hi, h_size, activation= tf.nn.tanh,
                                name='input_layer1')
            for i in range(self.u_layer):
                if i%2==0:
                    hi= tf.layers.dense(hi, h_size, activation= tf.nn.softplus,
                                        name='h_layer'+str(i))
                else:
                    hi= tf.sin(tf.layers.dense(hi, h_size), name='h_layer'+str(i))
            out= tf.layers.dense(hi, out_size, name='out_layer')
        return(out)


    def DNN_v(self, x_in, out_size, name, reuse):
        h_size= self.v_hidden_size
        with tf.variable_scope(name, reuse= reuse):
            hi= tf.layers.dense(x_in, h_size, activation= tf.nn.tanh,
                                name='input_layer')
            hi= tf.layers.dense(hi, h_size, activation= tf.nn.softplus,
                                name='input_layer1')
            for i in range(self.v_layer):
                if i%2==0:
                    hi= tf.layers.dense(hi, h_size, activation= tf.nn.softplus,
                                                            name='h_layer'+str(i))
                else:
                    hi= tf.sin(tf.layers.dense(hi, h_size), name='h_layer'+str(i))
            out= tf.layers.dense(hi, out_size, name='out_layer')
        return(out)


    def grad_u(self, x, out_size, name):
        #**************************************
        # u(x,y)
        fun_u= self.DNN_u(x, out_size, name, tf.AUTO_REUSE)   
        #*************************************
        # grad_u(x,y)
        grad_u= tf.gradients(fun_u, x, unconnected_gradients='zero')[0]
        return(fun_u, grad_u)


    def grad_v(self, x, out_size, name):
        #**************************************
        # v(x,y)
        fun_v= self.DNN_v(x, out_size, name, tf.AUTO_REUSE)
        #*************************************
        # grad_v(x,y)
        grad_v= tf.gradients(fun_v, x, unconnected_gradients='zero')[0]
        return(fun_v, grad_v)


    def fun_a(self, x):   ##PDE公式中的系数a
        #********************************************************
        out= 1.0
        return(out)


    def fun_w(self, x, low=0, up=1.0):    ##phi函数的一部分，phi=w*v，其中w在boundary为0
        I1= 0.210987
        x_list= tf.split(x, self.dim, 1)
        #**************************************************
        x_scale_list=[]

        for i in range(self.dim):
          x_scale= x_list[i]   ##做标准化处理
          x_scale_list.append(x_scale)
        #*************************************************
        z_x_list=[];
        for i in range(self.dim):
          supp_x= tf.greater(1-tf.abs(x_scale_list[i]), 0) & tf.greater(tf.abs(x_scale_list[i]),0)   ##假设w是距离函数
          z_x= tf.where(supp_x, tf.exp(1/(tf.pow(x_scale_list[i], 2)-1))/I1,   ##w在domain内有值，在boundary上为0
                     tf.zeros_like(x_scale_list[i]))  
          z_x_list.append(z_x)
        #***************************************************
        w_val= tf.constant(1.0)  ##创建常量
        for i in range(self.dim):
          w_val= tf.multiply(w_val, z_x_list[i])  #w在domain上有值，在boundary上为0
          dw= tf.gradients(w_val, x, unconnected_gradients='zero')[0]
          dw= tf.where(tf.is_nan(dw), tf.zeros_like(dw), dw)
        return(w_val, dw)


    def build(self):
        #**************************************************************
        with tf.name_scope('placeholder'):
            self.x_domain= tf.placeholder(tf.float32, shape=[None, self.dim], name='x_dm')
            self.f_obv= tf.placeholder(tf.float32, shape=[None, 1], name='f_obv')
            self.x_bound= tf.placeholder(tf.float32, shape=[None, self.dim], name='x_b')
            self.g_obv= tf.placeholder(tf.float32, shape=[None, 1], name='g_obv')
            self.int_domain= tf.placeholder(tf.float32, shape=(), name='int_domain')
        #**************************************************************
        name_u= 'dnn_u'; name_v= 'dnn_v';
        self.u_val, self.du= self.grad_u(self.x_domain, 1, name_u)
        self.v_val, self.dv= self.grad_v(self.x_domain, 1, name_v)
        self.w_val, self.dw= self.fun_w(self.x_domain)
        u_bound= self.DNN_u(self.x_bound, 1, name_u, tf.AUTO_REUSE)
        #
        a_val= self.fun_a(self.x_domain)
        self.wv= tf.multiply(self.w_val, self.v_val)
        #
        du_dw= tf.reduce_sum(tf.multiply(self.du, self.dw), axis=1)  
        du_dw= tf.reshape(du_dw, [-1,1])
        #
        du_dv= tf.reduce_sum(tf.multiply(self.du, self.dv), axis=1)
        du_dv= tf.reshape(du_dv, [-1,1])
        #
        du_dwv= tf.add(tf.multiply(self.v_val, du_dw),   ##分部计算dwv，du相当于已知
                       tf.multiply(self.w_val, du_dv))
        #**************************************************************
        with tf.name_scope('loss'):
            # 
            v_norm= tf.multiply(tf.reduce_mean((self.v_val)**2), self.int_domain)  ##phi的2范数的平方，只用了v是因为这里参数更新只与v有关
            #
            with tf.name_scope('u_loss'):
                #
                int_l1= tf.multiply(tf.reduce_mean(tf.multiply(
                    a_val,du_dwv)), self.int_domain)
                int_r= tf.multiply(tf.reduce_mean(
                    tf.multiply(self.f_obv, self.wv)), self.int_domain)  
                #
                int_l1_w= tf.multiply(tf.reduce_mean(tf.multiply(      ##loss_int_w只算了w没有v
                    a_val,du_dw)), self.int_domain)
                int_r_w= tf.multiply(tf.reduce_mean(
                    tf.multiply(self.f_obv, self.w_val)), self.int_domain)
                #
                self.loss_int= tf.square(int_l1-int_r) / v_norm  
                self.loss_int_w= tf.square(int_l1_w-int_r_w)                        
                # 
                self.loss_bound= tf.reduce_sum(tf.abs(u_bound-self.g_obv))   
                #
                self.loss_u= (50)*self.loss_bound+(1.0)*self.loss_int#+(0.0)*self.loss_int_w      ##为什么loss_int_w系数是0？？？
            with tf.name_scope('v_loss'):
                self.loss_v=  - tf.log(self.loss_int) 
        #**************************************************************
        # 
        u_vars= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='dnn_u')
        v_vars= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='dnn_v')
        #***************************************************************
        # 
        with tf.name_scope('optimizer'):
            self.u_opt= tf.train.AdagradOptimizer(self.u_rate).minimize(       ##优化
                    self.loss_u, var_list= u_vars)
            self.v_opt= tf.train.AdagradOptimizer(self.v_rate).minimize(
                    self.loss_v, var_list= v_vars)


    def train(self):
        #****************************************************************
        tf.reset_default_graph(); self.build(); 
        #***************************************************************
        mesh, test_x, test_u= self.sample_test(self.test_size, self.dim);    ##sample test函数的输出
        step=[]; error_l2r=[]; error_l2=[]; 
        time_begin=time.time(); time_list=[]; iter_time_list=[]
        #***************************************************************
        #saver= tf.train.Saver()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            #****************************
            for i in range(self.iteration):
                # mini-batch data
                train_data= self.sample_train(self.batch_size, self.bound_size, self.dim)  
                feed_train={self.x_domain: train_data[0],    ####feed_dict的作用是给使用placeholder创建出来的tensor赋值
                            self.f_obv: train_data[1],
                            self.x_bound: train_data[2],
                            self.g_obv: train_data[3],
                            self.int_domain: train_data[4]}
                if i%5==0: 
                    #*********************************
                    pred_u, pred_v= sess.run([self.u_val, self.v_val],feed_dict={self.x_domain: test_x})   ##用test_x得到新的u_val,v_val输出
                    err_l2= np.sqrt(np.mean(np.square(test_u-pred_u)))   ##pred_u是按神经网络方法计算得到的u
                    u_norm= np.sqrt(np.mean(np.square(test_u)))   ##u的范数吗？？？
                    step.append(i+1); 
                    error_l2r.append(err_l2/u_norm); error_l2.append(err_l2)  ##error_12r比error_12多除了一个u_norm
                    time_step= time.time(); time_list.append(time_step-time_begin)            
                if i%200==0: 
                    loss_v, loss_u, int_u, intw_u, loss_bd= sess.run(
                        [self.loss_v, self.loss_u, self.loss_int, self.loss_int_w, self.loss_bound], 
                        feed_dict= feed_train)
                    print('Iterations:{}'.format(i))
                    print('u_loss:{} v_loss:{}'.format(loss_u, loss_v))
                    print('int_u:{} int_u_w:{} loss_bd:{} err_l2r:{}'.format(
                        int_u, intw_u, loss_bd, error_l2r[-1]))  ##[-1]取最后一个元素
                    #self.show_error(step, error_l2r, self.dim, 'l2r')  ##show error什么意思？？？
                    #self.show_v_val(mesh, test_x, pred_v, 'v', i)
                    #self.show_u_val(mesh, test_x, test_u, pred_u, 'u', i)
                #
                iter_time0= time.time()
                for _ in range(self.v_step):  ##v_step步长
                    _ = sess.run(self.v_opt, feed_dict=feed_train)
                for _ in range(self.u_step):
                    _ = sess.run(self.u_opt, feed_dict=feed_train)
                iter_time_list.append(time.time()-iter_time0)
                #
            #*******************************************
            #self.show_error_abs(mesh, test_x, np.abs(test_u-pred_u), self.dim)
            #self.show_u_val(mesh, test_x, test_u, pred_u, 'u', i)
            print('L2_e is {}, L2_re is {}'.format(np.min(error_l2), np.min(error_l2r)))
            print('Running time is:{}'.format(time_list[-1]))
        return(mesh, test_x, test_u, pred_u, step, error_l2, error_l2r, time_list, iter_time_list, self.dim)



In [6]:
if __name__=='__main__':
    demo= pde_wan()
    mesh, test_x, test_u, pred_u, step, error_l2, error_l2r, time_list, iter_time_list, dim = demo.train()
    #***************************
    # save data as .mat form
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir('/content/drive/My Drive/Colab_Notebooks')
    file_path= './problem_data/'   #文件路径为当前运行目录下的/problem_data/

    import scipy.io
    data_save= {}
    data_save['mesh']= mesh  ##是一个list，里面有两个矩阵，矩阵的元素是将domain划分好的x取值点，
    data_save['test_x']= test_x  ##在domain内x的取值点，由mesh而来
    data_save['test_u']= test_u  ##由test_x得到的u值
    data_save['pred_u']= pred_u  ##由test_x通过神经网络得到的u值
    data_save['step']= step   ##每隔5次迭代输出的次数
    data_save['error_l2']= error_l2   ##test_u,pred_u的差
    data_save['error_l2r']= error_l2r   ##比error_12r多除一个u_norm
    data_save['time_list']= time_list   ##每5次迭代的总运行时间
    data_save['iter_time_list']= iter_time_list  ##每次迭代的总运行时间
    scipy.io.savemat(file_path+'tensorflow1_rectangular_plate', data_save)
    print('Data saved in '+file_path)

Iterations:0
u_loss:21784.083984375 v_loss:-0.291059285402298
int_u:1.337843894958496 int_u_w:1.305614709854126 loss_bd:435.6549072265625 err_l2r:5.3667378425598145
Iterations:200
u_loss:481.8812255859375 v_loss:3.8069732189178467
int_u:0.02221531979739666 int_u_w:0.02213660255074501 loss_bd:9.63718032836914 err_l2r:0.21286655962467194
Iterations:400
u_loss:399.6933898925781 v_loss:2.8527276515960693
int_u:0.057686757296323776 int_u_w:0.03009248897433281 loss_bd:7.9927144050598145 err_l2r:0.18609550595283508
Iterations:600
u_loss:408.1209411621094 v_loss:-0.32916563749313354
int_u:1.3898080587387085 int_u_w:0.024208970367908478 loss_bd:8.134622573852539 err_l2r:0.18019478023052216
Iterations:800
u_loss:400.6744689941406 v_loss:-0.8960170149803162
int_u:2.4498260021209717 int_u_w:0.02610190212726593 loss_bd:7.9644927978515625 err_l2r:0.17642946541309357
Iterations:1000
u_loss:341.39849853515625 v_loss:0.733065664768219
int_u:0.4804338812828064 int_u_w:0.027306735515594482 loss_bd:6.8183

In [ ]:
demo=pde_wan()
x_domain= np.random.uniform(0, 1, [300, 2])
name_u= 'dnn_u'
u,du=demo.grad_u(x_domain,1,name_u)
print(du)
print(u)

Tensor("zeros_like_8:0", shape=(300, 2), dtype=float64)
Tensor("dnn_u_8/out_layer/BiasAdd:0", shape=(300, 1), dtype=float64)
